### This notebook is an exemplar which demonstrates transferring files from a Box folder to a users home directory on Savio. 

( tested with boxsdk (2.0.0a2) on python 3.5 kernel)
pip install -Iv boxsdk==2.0.0a2 

function to store the oauth2 refresh token in a local file. This can be modified to use a keychain or other as required.

In [5]:
def store_tokens(access_token, refresh_token):
    
    """Callback for storing refresh tokens. (For now we ignore access tokens)."""
    with open('apptoken.cfg', 'w') as f:
     f.write(refresh_token.strip())

Oauth2 information is read from a local file with three lines, one line per parameter. 
The client id and client secret are defined in the globus application created for this notebook.
The redirect uri can be any site that requires validation. Run the bootstrap notebook to create initial 
tokens that are then continually refreshed.

In [6]:
import os

CLIENT_ID = None
CLIENT_SECRET = None
REDIRECT_URI = None
os.chdir('/global/home/users/mmanning')
# Read app info from text file
with open('app.cfg', 'r') as app_cfg:
    CLIENT_ID = app_cfg.readline()
    CLIENT_SECRET = app_cfg.readline()
    REDIRECT_URI = app_cfg.readline()


The refresh token is read from a local file.
This token was created by running the bootstrap notebook which requires the user to validate
with CalNet Authentication Service credentials, then stores the returned auth and refresh tokens 
in the same config files.

In [7]:
REFRESH_TOKEN = None

# Read app info from text file
with open('apptoken.cfg', 'r') as apptoken_cfg:
    REFRESH_TOKEN = apptoken_cfg.readline()

Perform autentication then create globus client
Verify client is working by retrieving the name of the users root folder in Box

In [8]:
from boxsdk import OAuth2
from boxsdk import Client

# Do OAuth2 authorization.
oauth = OAuth2(
    client_id=CLIENT_ID.strip(),
    client_secret=CLIENT_SECRET.strip(),
    refresh_token=REFRESH_TOKEN.strip(),
    store_tokens=store_tokens
)

client = Client(oauth)

root_folder = client.folder(folder_id='0').get()
print ("folder name: ", root_folder['name'] )

items = client.folder(folder_id='0').get_items(limit=100, offset=0)
#print ("items: ", items )



folder name:  All Files


get the items from the base folder. currently the Box SDK does not have an option for finding a folder by name so if you are looking for a specific folder then you would need to loop thru all the items in the list below and do a name match. Once you find the folder and retrieve the id, you can save that id for subsequent runs. I have not found a way to get the id of the folder from the Box web client.

In [12]:
import os
import shutil 

print ('current working directory: ', os.getcwd())
os.chdir('/global/scratch/mmanning/test')

# test folder contents
items = client.folder(folder_id='11943857315').get_items(limit=20, offset=0)
if type(items) is list:
    print ('number of files in top folder: ', len(items) )
    
    for item in items:
        print('item name: ', item['name'])
        print('  -- item id: ', item['id'])
        
        if item['type'] == 'folder':
            print('folder name: ', item['name'])
            print('folder id: ', item['id'])
        
        if  not item['type'] == 'folder' and item['name'].endswith('.jpg'):
            imagecontent = client.file(file_id=item['id']).content()
            newfile = open('/global/scratch/mmanning/' + item['name'], 'wb')
            newfile.write(imagecontent)
            newfile.close()
            
    

current working directory:  /global/scratch/mmanning/test
number of files in top folder:  2
item name:  IMG_20160922_124853.jpg
  -- item id:  100082909799
item name:  IMG_20160922_125044.jpg
  -- item id:  100084250900


Create a new folder in the base directory and upload image files in the current folder.

In [19]:
newFolder = client.folder(folder_id='0').create_subfolder('ThisIsATest')
newFolderId = newFolder['id']
sourceFolder = '/global/scratch/mmanning/'

print ("new folder id: ", newFolderId )

upload_folder = client.folder(folder_id=newFolderId).get()

# upload all the files in the current folder  if os.path.isfile(f)
files = [f for f in os.listdir(sourceFolder) ] 
print ("files: ", files )

for filename in files:
    print ("file name: ", filename ) 
    
    if filename.endswith('.jpg'):
        upload_folder.upload(sourceFolder + filename)
    
#    if filename.endswith('.jpg'):
#        with open(filename, 'r') as fileobj:
#            upload_folder.upload(fileobj)


new folder id:  11944380587
files:  ['IMG_20160922_125044.jpg', 'xserver2', 'boxDump', 'photoscan', 'lftpTests', 'missing', 'logs', 'IMG_20160922_124853.jpg', 'hawkins1988.PDF', 'sparkTest', 'jupyterTest3.img', 'test', 'testSav', 'jhubtest', 'cat1ls.log', 'photoscantest', 'spark', 'xserver', 'photoscantest1.log', 'pythonNotebooks', 'globustest', '3834_237_000.tif', 'tesseract', 'lftp.sh', 'test.tif']
file name:  IMG_20160922_125044.jpg
file name:  xserver2
file name:  boxDump
file name:  photoscan
file name:  lftpTests
file name:  missing
file name:  logs
file name:  IMG_20160922_124853.jpg
file name:  hawkins1988.PDF
file name:  sparkTest
file name:  jupyterTest3.img
file name:  test
file name:  testSav
file name:  jhubtest
file name:  cat1ls.log
file name:  photoscantest
file name:  spark
file name:  xserver
file name:  photoscantest1.log
file name:  pythonNotebooks
file name:  globustest
file name:  3834_237_000.tif
file name:  tesseract
file name:  lftp.sh
file name:  test.tif
